# 🍎 Chat Completions with AIProjectClient 🍏

In this notebook, we'll demonstrate how to perform **Chat Completions** using the **Azure AI Foundry** SDK. We'll combine **`azure-ai-projects`** and **`azure-ai-inference`** packages to:

1. **Initialize** an `AIProjectClient`.
2. **Obtain** a Chat Completions client to do direct LLM calls.
3. **Use** a **prompt template** to add system context.
4. **Send** user prompts in a health & fitness theme.

## 🏋️ Health-Fitness Disclaimer
> **This example is for demonstration only and does not provide real medical advice.** Always consult a professional for health or medical-related questions.

### Prerequisites
Before starting this notebook, please ensure you have completed all prerequisites listed in the root [README.md](../../README.md#-prerequisites).

Let's get started! 🎉

<img src="./seq-diagrams/1-chat.png" width="75%%"/>


## 1. Initial Setup
Load environment variables, create an `AIProjectClient`, and fetch a [`ChatCompletionsClient`](https://learn.microsoft.com/en-au/azure/ai-foundry/foundry-models/how-to/use-chat-completions?tabs=python#use-the-chat-completions-api). We'll also define a **prompt template** to show how you might structure a system message.
Review [prompt engineering techniques](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/concepts/prompt-engineering?context=%2Fazure%2Fai-foundry%2Fcontext%2Fcontext) for additional insights and patterns.


In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import AzureCliCredential, InteractiveBrowserCredential, ChainedTokenCredential

from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Retrieve from environment
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")
tenant_id = os.environ.get("TENANT_ID")

try:
    # Create tenant-specific credential to bypass Azure CLI cache issues
    credential = ChainedTokenCredential(
        AzureCliCredential(tenant_id=tenant_id),
        InteractiveBrowserCredential(tenant_id=tenant_id)
    )
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)

✅ Successfully created AIProjectClient


### Prompt Template
We'll define a quick **system** message that sets the context as a friendly, disclaimer-providing fitness assistant.

```txt
SYSTEM PROMPT (template):
You are FitChat GPT, a helpful fitness assistant.
Always remind users: I'm not a medical professional.
Be friendly, provide general advice.
...
```

We'll then pass user content as a **user** message.


In [2]:
# We'll define a function that runs chat completions with a system prompt & user prompt
def chat_with_fitness_assistant(user_input: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    system_text = (
        "You are FitChat GPT, a friendly fitness assistant.\n"
        "Always remind users: I'm not a medical professional.\n"
        "Answer with empathy and disclaimers."
    )

    # We'll open the Azure OpenAI client
    with project_client.get_openai_client(api_version="2024-10-21") as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=user_input)

        # Send the request
        response = chat_client.chat.completions.create(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content  # simplest approach: get top choice's content

print("Defined a helper function to do chat completions.")

Defined a helper function to do chat completions.


## 2. Try Chat Completions 🎉
We'll call the function with a user question about health or fitness, and see the result. Feel free to modify the question or run multiple times!


In [3]:
user_question = "How can I start a beginner workout routine at home?"
reply = chat_with_fitness_assistant(user_question)
print("🗣️ User:", user_question)
print("🤖 Assistant:", reply)

🗣️ User: How can I start a beginner workout routine at home?
🤖 Assistant: Starting a workout routine at home is a fantastic decision, and I’m excited to help guide you! Remember to take it at your own pace, listen to your body, and don’t hesitate to adjust based on your comfort level. Also, always check with your doctor before starting any new exercise routine, especially if you have any medical conditions or concerns—I'm not a medical professional, so safety first!

### Here’s a simple step-by-step guide for getting started:

---

### 1. **Set Clear and Realistic Goals**
   - Ask yourself: Why do you want to start exercising? (e.g., gain strength, lose weight, boost energy, improve mental health). This clarity will help you stay motivated.
   - Start small—aim for consistency first (e.g., 20-30 minutes, 3-4 times a week).

---

### 2. **Create a Comfortable Workout Space**
   - Find an area with enough room to move freely.
   - Use household items if you don’t have equipment! For exam

## 3. Another Example: Prompt Template with Fill-Ins 📝
We can go a bit further and add placeholders in the system message. For instance, imagine we have a **userName** or **goal**. We'll show a minimal example.


In [4]:
def chat_with_template(user_input: str, user_name: str, goal: str):
    # Construct a system template with placeholders
    system_template = (
        "You are FitChat GPT, an AI personal trainer for {name}.\n"
        "Your user wants to achieve: {goal}.\n"
        "Remind them you're not a medical professional. Offer friendly advice."
    )

    # Fill in placeholders
    system_prompt = system_template.format(name=user_name, goal=goal)

    with project_client.get_openai_client(api_version="2024-10-21") as chat_client:
        system_msg = SystemMessage(content=system_prompt)
        user_msg = UserMessage(content=user_input)

        response = chat_client.chat.completions.create(
            model=model_deployment,
            messages=[system_msg, user_msg]
        )

    return response.choices[0].message.content

# Let's try it out
templated_user_input = "What kind of home exercise do you recommend for a busy schedule?"
assistant_reply = chat_with_template(
    templated_user_input,
    user_name="Jordan",
    goal="increase muscle tone and endurance"
)
print("🗣️ User:", templated_user_input)
print("🤖 Assistant:", assistant_reply)

🗣️ User: What kind of home exercise do you recommend for a busy schedule?
🤖 Assistant: That’s a great goal, Jordan! I’m glad you’re looking to fit exercise into your busy schedule. Just a quick reminder that I’m not a medical professional—so always listen to your body and consult a doctor if needed. Here’s a simple and effective home workout plan that focuses on muscle tone and endurance. You can do it in as little as 20–30 minutes and no equipment is needed (but feel free to use dumbbells if you have them):  

---

### **Warm-Up (3–5 minutes)**  
- **Jumping Jacks**: 1 minute to get your heart rate up.  
- **Arm Circles + High Knees**: 30 seconds arm circles forward + 30 seconds backward while alternating high knees.  
- **Dynamic Stretching**: Stretch your major muscle groups with moving stretches like walking lunges or leg swings.  

---

### **Circuit Workout (20–25 minutes)**  
Do this circuit 2–3 times, depending on the time you have.

1. **Squats** (Bodyweight or Weighted): 12–1

## 🎉 Congratulations!
You've successfully performed **chat completions** with the Azure AI Foundry's `AIProjectClient` and `azure-ai-inference`. You've also seen how to incorporate **prompt templates** to tailor your system instructions.

#### Head to [2-embeddings.ipynb](2-embeddings.ipynb) for the next part of the workshop! 🎯